In [5]:
from vercast.pfc import *
from vercast.middle_term import build_regressors
import pandas as pd
import psycopg2
import plotly.graph_objects as go
from collect_transform_data import execute_query

# 1. Import data

## 1.1 Queries

In [6]:
years = (2017, 2018, 2019)

In [10]:
query_spot = f"""
SELECT
    delivery_start AS timestamp,
    price AS spot_price
FROM elec_day_ahead_market
WHERE tenor = 'PT1H' 
AND country = 'FR'
AND EXTRACT(YEAR FROM delivery_start) IN {years} --EXTRACT(YEAR FROM (delivery_start AT TIME ZONE 'UTC' AT TIME ZONE 'CET')) IN {years}
ORDER BY timestamp;
"""

In [11]:
query_load = f"""
SELECT
    date_trunc('hour', cons_start) AS timestamp,
    AVG(quantity) AS load
FROM consumption
WHERE source = 'RTE' 
AND country = 'FR'
AND curve_type = 'REALISED'
AND EXTRACT(YEAR FROM cons_start) IN {years} --EXTRACT(YEAR FROM (cons_start AT TIME ZONE 'UTC' AT TIME ZONE 'CET')) IN {years}
GROUP BY timestamp
ORDER BY timestamp;
"""

In [12]:
query_pv = f"""
SELECT
    prod_start AS timestamp,
    quantity AS pv
FROM production_per_type
WHERE tenor = 'PT1H' 
AND country = 'FR'
AND EXTRACT(YEAR FROM prod_start) IN {years} --EXTRACT(YEAR FROM (prod_start AT TIME ZONE 'UTC' AT TIME ZONE 'CET')) IN {years}
AND production_type = 'SOLAR'
ORDER BY timestamp;
"""

In [13]:
query_pv_cf = f"""
SELECT 
    p.prod_start AS timestamp,
    pv / pv_capa AS pv_cf
FROM (
    SELECT 
        prod_start, 
        prod_end, 
        quantity AS pv
    FROM production_per_type 
    WHERE tenor = 'PT1H' 
    AND country = 'FR' 
    AND production_type = 'SOLAR'
    ) AS p 
JOIN (
    SELECT 
        capa_start, 
        capa_end, 
        quantity AS pv_capa 
    FROM installed_capacities 
    WHERE capacities_type = 'SOLAR' 
    AND country = 'FR'
    ) AS c
ON c.capa_start <= p.prod_start 
AND c.capa_end >= p.prod_end
WHERE EXTRACT(YEAR FROM p.prod_start) IN {years} --EXTRACT(YEAR FROM (p.prod_start AT TIME ZONE 'UTC' AT TIME ZONE 'CET')) IN {years}
ORDER BY timestamp;
"""

In [14]:
query_wind = f"""
SELECT
    prod_start AS timestamp,
    SUM(quantity) AS wind
FROM production_per_type
WHERE tenor = 'PT1H' 
AND country = 'FR'
AND EXTRACT(YEAR FROM prod_start) IN {years} --EXTRACT(YEAR FROM (prod_start AT TIME ZONE 'UTC' AT TIME ZONE 'CET')) IN {years}
AND production_type IN ('WIND_OFFSHORE', 'WIND_ONSHORE')
GROUP BY timestamp
ORDER BY timestamp;
"""

In [18]:
query_wind_cf = f"""
SELECT 
    p.prod_start AS timestamp,
    wind / wind_capa AS wind_cf
FROM (
    SELECT 
        prod_start, 
        prod_end, 
        SUM(quantity) AS wind
    FROM production_per_type 
    WHERE tenor = 'PT1H' 
    AND country = 'FR' 
    AND production_type IN ('WIND_OFFSHORE', 'WIND_ONSHORE')
    GROUP BY (prod_start, prod_end)
    ) AS p 
JOIN (
    SELECT 
        capa_start, 
        capa_end, 
        SUM(quantity) AS wind_capa 
    FROM installed_capacities 
    WHERE capacities_type IN ('WIND_OFFSHORE', 'WIND_ONSHORE')
    AND country = 'FR'
    GROUP BY (capa_start, capa_end)
    ) AS c
ON c.capa_start <= p.prod_start 
AND c.capa_end >= p.prod_end
WHERE EXTRACT(YEAR FROM p.prod_start) IN {years} --EXTRACT(YEAR FROM (p.prod_start AT TIME ZONE 'UTC' AT TIME ZONE 'CET')) IN {years}
ORDER BY timestamp;
"""

In [16]:
query_temp = f"""
SELECT
    date_trunc('hour', temp_start) AS timestamp,
    AVG(tn) AS tn,
    AVG(tr) AS tr, 
    AVG(trs) AS trs, 
    AVG(delta_t) as delta_t, 
    AVG(delta_ts) AS delta_ts
FROM temperature
WHERE tenor = 'PT30M'
AND country = 'FR'
AND source = 'ENEDIS'
AND EXTRACT(YEAR FROM temp_start) IN {years} --EXTRACT(YEAR FROM (temp_start AT TIME ZONE 'UTC' AT TIME ZONE 'CET')) IN {years}
GROUP BY timestamp
ORDER BY timestamp;
"""

## 1.2 Execute queries

In [19]:
spot_train = execute_query(query_spot, timestamp=True).astype(float)
temp_train = execute_query(query_temp, timestamp=True).astype(float).round(2)
load_train = execute_query(query_load, timestamp=True).astype(float).round(2)
pv_train = execute_query(query_pv, timestamp=True).astype(float).resample('h').ffill()
pv_cf_train = execute_query(query_pv_cf, timestamp=True).astype(float).resample('h').ffill()
eol_train = execute_query(query_wind, timestamp=True).astype(float).resample('h').ffill()
eol_cf_train = execute_query(query_wind_cf, timestamp=True).astype(float).resample('h').ffill()

In [21]:
normalized_load_train = load_train.groupby(load_train.index.year).transform(lambda x: x/x.sum())

In [22]:
regressors = build_regressors(
    timestamp=spot_train.index,
    deltaTS=temp_train['delta_ts'],
    load=load_train,
    pv=pv_train,
    eol=eol_train)

normalized_regressors = build_regressors(
    timestamp=spot_train.index,
    deltaTS=temp_train['delta_ts'],
    load=normalized_load_train,
    pv_cf=pv_cf_train,
    eol_cf=eol_cf_train,
    days=False, months=False, hours=False)

# 2. Calibration

In [23]:
inputs_cal = {}
inputs_cal["eval_years"] = [2026]
inputs_cal['name'] = 'v1'
inputs_cal['residual_model_config'] = {'n_states' : 80, 'isweekend' : False}
inputs_cal['regressor_model_config'] = {'n_states' : 20, 'isweekend' : True}
inputs_cal['nscenario'] = 10
inputs_cal['output_path'] = os.path.join("/frontend/PFC/outputs")
inputs_cal['save_files'] = True
inputs_cal['sampling_period'] = 'W'

In [24]:
calibrate(spot=spot_train,
          regressors=regressors,
          normalized_regressors=normalized_regressors,
          **inputs_cal)

# 3. Normalized PFC

In [26]:
inputs_gen = {}
inputs_gen['calibration_path'] = '/frontend/PFC/outputs/2025-02-06_v1'
inputs_gen['generation_id'] = 'BP23'
inputs_gen['scenario_file'] = '/frontend/vercast/Scénarios variables explicatives.xlsx'
inputs_gen["pv_scenario_id"] = "RTE - référence - BP23"
inputs_gen["eol_scenario_id"] = "RTE - référence - BP23"
inputs_gen["load_scenario_id"] = "RTE - référence - BP23"
inputs_gen["sampling_period"] = 'W'

In [27]:
norm_PFC = generate_normalized_pfc(**inputs_gen, save_files=True)

[[1.000e+00 4.710e+08 3.005e+04 3.195e+04]
 [1.000e+00 4.710e+08 3.005e+04 3.195e+04]
 [1.000e+00 4.710e+08 3.005e+04 3.195e+04]
 ...
 [1.000e+00 4.710e+08 3.005e+04 3.195e+04]
 [1.000e+00 4.710e+08 3.005e+04 3.195e+04]
 [1.000e+00 4.710e+08 3.005e+04 3.195e+04]]


In [28]:
norm_PFC

,0,1,2,3,4,5,6,7,8,9
2026-01-01 00:00:00,1.068858,1.051883,1.184184,1.079966,1.107931,1.004440,1.161718,1.355885,1.228348,1.176111
2026-01-01 01:00:00,0.980482,1.075710,1.211008,1.104429,1.133028,1.027192,1.188033,1.386598,1.126785,1.202752
2026-01-01 02:00:00,0.945926,1.029137,1.158578,1.056613,1.083974,0.982720,1.136598,1.326566,1.087073,1.150679
2026-01-01 03:00:00,0.949597,0.918518,1.171122,1.068053,0.967461,0.993360,1.032096,1.204598,1.072611,1.181024
2026-01-01 04:00:00,0.965500,0.926929,1.222437,1.108203,0.976319,1.030702,1.049381,1.224771,1.260465,1.240084
...,...,...,...,...,...,...,...,...,...,...
2026-12-31 19:00:00,1.935277,0.813226,1.198755,0.125913,0.538269,0.628101,1.012670,0.897612,0.539141,1.313521
2026-12-31 20:00:00,1.774297,0.795771,1.062199,0.029537,0.480841,0.650772,0.934941,0.583837,0.263973,1.377526
2026-12-31 21:00:00,1.713793,0.798350,1.153238,0.163164,0.515738,0.688420,0.935007,0.416110,0.239225,1.357996
2026-12-31 22:00:00,1.604347,0.809017,1.114887,0.207420,0.288618,0.814090,0.901502,0.542553,0.103337,1.315798


# 4. Adjust PFC

In [47]:
inputs_adj = {}
inputs_adj['historic_years'] = [2017, 2018, 2019]
inputs_adj['adjustment_date'] = '2024-09-18'
inputs_adj['eex_path'] = '/frontend/vercast/EEX'
inputs_adj['calibration_path'] = '/frontend/PFC/outputs/2025-02-06_v1'
inputs_adj['generation_id'] = 'BP23'
inputs_adj['sampling_period'] = 'W'

In [48]:
adj_PFC = adjust_pfc(**inputs_adj, save_files=True)

FileNotFoundError: [Errno 2] No such file or directory: '/frontend/PFC/outputs/2025-02-06_v1/BP23/normalized_pfc.csv'